In [4]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [5]:
from companies.models import *
Company.objects.create(name="Company AAA", num_employees=120, num_chairs=150, num_tables=60)
Company.objects.create(name="Company BBB", num_employees=50, num_chairs=30, num_tables=20)
Company.objects.create(name="Company CCC", num_employees=100, num_chairs=40, num_tables=40)

<Company: Company CCC>

In [6]:
from django.db.models import Count, F, Value
from django.db.models.functions import Length, Upper
from django.db.models.lookups import GreaterThan

In [7]:
Company.objects.filter(num_employees__gt=F("num_chairs"))

<QuerySet [<Company: Company BBB>, <Company: Company CCC>]>

In [8]:
Company.objects.filter(num_employees__gt=F("num_chairs") * 2)

<QuerySet [<Company: Company CCC>]>

In [9]:
Company.objects.filter(num_employees__gt=F("num_chairs") + F("num_tables"))

<QuerySet [<Company: Company CCC>]>

In [10]:
company = (
    Company.objects.filter(num_employees__gt=F("num_chairs"))
    .annotate(chairs_needed=F("num_employees") - F("num_chairs"))
    .first()
)

In [11]:
company.name

'Company BBB'

In [12]:
company.chairs_needed

20

In [13]:
from django.db.models import CharField, Value as V
from django.db.models.functions import Concat

In [14]:
company = (
    Company.objects.annotate(
        fullname=Concat("name", V(" ("), "ticker", V(")"))
    )
)

for com in company:
    print(com.fullname)

Company AAA ()
Company BBB ()
Company CCC ()


In [16]:
from books.models import *

In [17]:
Book.objects.count()

59

In [18]:
Book.objects.filter(publisher__name="Penguin Books").count()

20

In [19]:
from django.db.models import Avg

In [20]:
Book.objects.aggregate(Avg("price", default=0))

{'price__avg': Decimal('9.7018644067796610')}

In [21]:
from django.db.models import Max

In [22]:
Book.objects.aggregate(Max("price", default=0))

{'price__max': Decimal('14.99')}

In [24]:
from books.models import Publisher
from django.db.models import Count

In [25]:
pubs = Publisher.objects.annotate(num_books=Count("book"))

In [28]:
pubs[0].num_books

20

In [29]:
from django.db.models import Q
above = Publisher.objects.annotate(above_4=Count("book", filter=Q(book__rating__gt=4)))
below = Publisher.objects.annotate(below_4=Count("book", filter=Q(book__rating__lte=4)))
above[0].above_4

16

In [30]:
below[0].below_4

4

In [31]:
pubs = Publisher.objects.annotate(num_books=Count("book")).order_by("-num_books")[:5]

In [32]:
pubs[0].num_books

39